In [1]:
# %load /home/shleifer/durbango/imports.py
#IMPORTS

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ISO = "ISO-8859-1"

import os
import pickle
from tqdm import tqdm, tqdm_notebook, tnrange
import numpy as np
import pandas as pd
import gc
import IPython
import sys

from sklearn.metrics import *
import itertools
from collections import *
from tqdm import tqdm, tqdm_notebook
import funcy
import matplotlib.pyplot as plt
from time import time
from sklearn.linear_model import *
from functools import partial
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, KFold, ParameterGrid
from sklearn.utils import shuffle
from numpy.testing import assert_array_equal
from pandas.testing import assert_frame_equal, assert_series_equal, assert_index_equal
from sklearn.preprocessing import StandardScaler, RobustScaler
from pathlib import Path, PosixPath
import re
from glob import glob
import gzip
#from pandas_summary import DataFrameSummary
import pickle
from IPython.lib.display import FileLink

mock_arr = np.array(range(25)).reshape(5,5)
mock_df = pd.DataFrame(np.ones((5,5)), columns=['a', 'b', 'c', 'd', 'e'])
mock_df['color'] = ['red', 'blue', 'red', 'blue', 'green']
mock_df['bool_col'] = [True, False, True, False, True]


np.set_printoptions(precision=5, linewidth=110, suppress=True)

from ipykernel.kernelapp import IPKernelApp
def in_notebook(): return IPKernelApp.initialized()
def printmd(string):
    display(Markdown(string))
from IPython.display import Markdown, display, HTML
from IPython.core.interactiveshell import InteractiveShell

display(HTML("<style>.container { width:90% !important; }</style>"))

# pretty print only the last output of the cell
# InteractiveShell.ast_node_interactivity = "last_expr" # "all" for all
try:
    import eli5
    from lightgbm.sklearn import LGBMClassifier, LGBMRegressor
    import seaborn as sns
except ImportError:
    pass
def pickle_save(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)
def pickle_load_gzip(path):
    with gzip.open(path, 'rb') as f:
        return pickle.load(f, encoding='latin-1')
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
save_pickle = pickle_save
load_pickle = pickle_load

DURBANGO_ATTEMPTS = [
    '/Users/shleifer/durbango/',
    '/home/paperspace/durbango/',
    '/home/shleifer/durbango/'
]
for DURBANGO_DIR in DURBANGO_ATTEMPTS:
    if Path(DURBANGO_DIR).exists():
        sys.path.append(DURBANGO_DIR)
        try:
            from durbango.nb_utils import *
            from durbango.tensorboard_parser import parse_tf_events_file
        except ImportError as e:
            print(f'Cant import durbango-- {e}')
        break


Path.ls =  property(lambda self: list(self.iterdir()))
ParameterGrid.l = property(lambda self: list(self))



In [2]:
paths = sorted(list(Path('dataset_distillation/results/distill_basic/').rglob('*.pth'))); paths

[PosixPath('dataset_distillation/results/distill_basic/I2/arch(AlexCifarNet,xavier,1.0)_distillLR0.001_E(400,40,0.5)_lr0.01_B1x10x3_train(unknown_init)/checkpoints/epoch0080/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/I2/arch(AlexCifarNet,xavier,1.0)_distillLR0.001_E(400,40,0.5)_lr0.01_B1x10x3_train(unknown_init)/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/I2/arch(DXResNet18,xavier,1.0)_distillLR0.02_E(400,40,0.5)_lr0.01_B1x1x3_train(known_init)/checkpoints/epoch0000/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/MNIST/arch(LeNet,xavier,1.0)_distillLR0.02_E(400,40,0.5)_lr0.01_B1x1x3_train(known_init)/checkpoints/epoch0000/results.pth')]

In [11]:
list(Path('.').rglob('*.pth'))

[PosixPath('dataset_distillation/results/distill_basic/MNIST/arch(LeNet,xavier,1.0)_distillLR0.02_E(400,40,0.5)_lr0.01_B1x1x3_train(known_init)/checkpoints/epoch0000/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/I2/arch(DXResNet18,xavier,1.0)_distillLR0.02_E(400,40,0.5)_lr0.01_B1x1x3_train(known_init)/checkpoints/epoch0000/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/I2/arch(AlexCifarNet,xavier,1.0)_distillLR0.001_E(400,40,0.5)_lr0.01_B1x10x3_train(unknown_init)/results.pth'),
 PosixPath('dataset_distillation/results/distill_basic/I2/arch(AlexCifarNet,xavier,1.0)_distillLR0.001_E(400,40,0.5)_lr0.01_B1x10x3_train(unknown_init)/checkpoints/epoch0080/results.pth'),
 PosixPath('docs_src/points.pth'),
 PosixPath('data/adult_sample/models/mini_train.pth')]

In [3]:
pth = paths[0]

In [4]:
import torch
results = torch.load(pth, map_location='cpu')

In [5]:
for a,b,c in results: 
    print(a.shape, b, c)
    break

torch.Size([10, 3, 32, 32]) tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) tensor([0.0587])


In [9]:
from fastai.script import *
from fastai.vision import *
from fastai.vision.models.xresnet2 import xresnet50_2
from fastai.vision.models.xresnet import xresnet50
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from fastai.imagito.utils import *
from fastai.imagito.classes import ClassUtils

ImportError: cannot import name 'ClassFolders' from 'fastai.imagito.classes' (/Users/shleifer/fastai-fork/fastai/imagito/classes.py)

In [7]:
import sys; sys.path.append('dataset_distillation/')

In [8]:
from dataset_distillation.utils.imagelists import *

In [9]:
import torchvision

In [13]:
save_dir = '/Users/shleifer/distilled_dataset_v0/'
if not Path(save_dir).exists():
    save_dir = '/home/shleifer/distilled_dataset_v0/'
save_distilled_images_for_fastai(pth, save_dir, map_location='cpu')

In [15]:
image_list = load_distilled_imagelist(save_dir)

/Users/shleifer/fastai-fork/fastai/data_block.py:455: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [ ]:
from fastai.vision.data import image_extensions, denormalize, imagenet_stats

In [ ]:
from PIL import Image

In [ ]:
#Image.fromarray(image2np(a[0]))

In [ ]:
img.shape

In [ ]:
lmap(np.array, imagenet_stats)

In [ ]:
img = denormalize(img, *lmap(torch.Tensor, imagenet_stats))

In [ ]:
img.mean()

In [ ]:
img.stdb()

In [ ]:
npy.shape

In [ ]:
torchvision.utils.save_image(img, 'test_tvsave.jpg')

In [ ]:
im = Image.open('test_tvsave.jpg')

In [ ]:
#npy = image2np(img)
Image.fromarray(img.numpy())